In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [3]:
# 通过selenium获取page_text
def getHTMLBySelenium(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe', options=chrome_options)
    browser.get(url)
    time.sleep(2)
    page_text = browser.page_source
    browser.quit()
    return page_text

In [13]:
# https://newmedia.chinacourt.org/vod/play/2021/03/07/12/68BA4C667E584FA09136352CE29483E2/chunklist_w981264293.m3u8
def getM3U8Url(url):
    page_text = getResponse(url).text
    tree = etree.HTML(page_text)
    ex = 'src: "https://newmedia.chinacourt.org/vod/play(.*?).m3u8"'
    result = re.findall(ex, page_text, re.S)
    playlist = 'https://newmedia.chinacourt.org/vod/play' + result[0] + '.m3u8'
    allContent = getResponse(playlist).text
    fileLine = allContent.split('\n')
    m3u8Url = playlist.rsplit('/', 1)[0] + '/' + fileLine[-2]
    return m3u8Url

In [14]:
print(getM3U8Url('https://tv.chinacourt.org/53755.html'))

https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/chunklist_w1309319220.m3u8


In [16]:
def downloadEveryTs(url):
    allContent = getResponse(url).text
    fileLine = allContent.split('\n')
    downloadPath = 'download'
    if not os.path.exists(downloadPath):
        os.mkdir(downloadPath)
    # 通过判断文件头来确定是否是M3U8文件
    if fileLine[0] != "#EXTM3U":
        #raise BaseException(u"非M3U8的链接")
        print('非M3U8的链接')
        return
    else:
        unknow = True# 用来判断是否找到了下载地址
        for index, line in enumerate(fileLine):
            if "EXTINF" in line:
                unknow = False
                # 拼出ts片段的URL
                tsUrl = url.rsplit('/', 1)[0] + '/' + fileLine[index + 1]# i.ts在下一行
                print(tsUrl)
                res = getResponse(tsUrl)
                cFuleName = (str(fileLine[index + 1])).split('?')[0]
                with open(downloadPath + '/' + cFuleName, 'ab') as fp:
                    fp.write(res.content)
                    fp.flush()
        if unknow:
            raise BaseException(u"未找到对应的下载地址")
        else:
            print("ts 下载完成")

In [17]:
downloadEveryTs('https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/chunklist_w1309319220.m3u8')

https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_0.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_1.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_2.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_3.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_4.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_5.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_6.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_7.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_8.ts
h

https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_74.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_75.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_76.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_77.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_78.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_79.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_80.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1309319220_81.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w130931922

In [27]:
# ts文件合并
def file_walker(path):
    file_list = []
    for root, dirs, files in os.walk(path): # 生成器
        for fn in files:
            p = str(root+'/'+fn)
            file_list.append(p)
 
    #print(file_list)
    return file_list
 
def combine(ts_path, combine_path, file_name):
    file_list = file_walker(ts_path)
    file_path = combine_path + file_name + '.ts'
    with open(file_path, 'wb+') as fw:
        for i in range(len(file_list)): 
            fw.write(open(file_list[i], 'rb').read())
            
# 删除之前下载的一段段的ts文件
def clear():
    shutil.rmtree('./download')
    time.sleep(1)
    os.mkdir('download')
    
def downloadTs(url, fileName):
    downloadEveryTs(url)
    combine('./download', 'ts/', fileName)
    print(fileName, ' ts successful')
    clear()

In [21]:
url = 'https://tv.chinacourt.org/53757.html'
page_text = getResponse(url).text
tree = etree.HTML(page_text)
allText = tree.xpath('//div[@class="player_txt"]//text()')
text = '\n'.join(allText)
print(text)


	
湖南省高级人民法院院长
田立文：
重大涉黑恶案件往往案情重大复杂，社会公众关注度高，必须更加严格审慎对待。湖南法院对该类案件高度重视，力求把每一次重大涉黑恶案件审判，既当成一次面向全省法院的庭审“示范课”，也当成一次面向社会的法治“公开课”。主要有以下几点做法。
 



In [22]:
def getText(url):
    page_text = getResponse(url).text
    tree = etree.HTML(page_text)
    allText = tree.xpath('//div[@class="player_txt"]//text()')
    text = '\n'.join(allText)
    return text

In [ ]:
# items
# https://tv.chinacourt.org/shipin-13.html?Video_page=6&ajax=yw1&page=6

In [24]:
url = 'https://tv.chinacourt.org/shipin-13.html?Video_page=6&ajax=yw1&page=6'
page_text = getResponse(url).text
tree = etree.HTML(page_text)
allLi = tree.xpath('//div[@class="items"]/li')
linkList = []
for li in allLi:
    href = li.xpath('.//a[1]/@href')[0]
    #print(href)
    linkList.append('https://tv.chinacourt.org' + href)

/215.html
/214.html
/213.html
/211.html
/212.html
/210.html
/209.html
/207.html
/162.html
/161.html
/160.html
/159.html
/158.html
/157.html
/143.html
/147.html
/142.html


In [25]:
def getPageUrl(url):
    page_text = getResponse(url).text
    tree = etree.HTML(page_text)
    allLi = tree.xpath('//div[@class="items"]/li')
    linkList = []
    for li in allLi:
        href = li.xpath('.//a[1]/@href')[0]
        #print(href)
        linkList.append('https://tv.chinacourt.org' + href)
    return linkList

In [26]:
for link in getPageUrl('https://tv.chinacourt.org/shipin-13.html?Video_page=6&ajax=yw1&page=6'):
    print(link)

https://tv.chinacourt.org/215.html
https://tv.chinacourt.org/214.html
https://tv.chinacourt.org/213.html
https://tv.chinacourt.org/211.html
https://tv.chinacourt.org/212.html
https://tv.chinacourt.org/210.html
https://tv.chinacourt.org/209.html
https://tv.chinacourt.org/207.html
https://tv.chinacourt.org/162.html
https://tv.chinacourt.org/161.html
https://tv.chinacourt.org/160.html
https://tv.chinacourt.org/159.html
https://tv.chinacourt.org/158.html
https://tv.chinacourt.org/157.html
https://tv.chinacourt.org/143.html
https://tv.chinacourt.org/147.html
https://tv.chinacourt.org/142.html


In [28]:
def downloadVideoAndText(url, fileName):
    text = getText(url)
    with open('text/' + fileName + '.txt', 'w', encoding='utf-8') as fp:
        fp.write(text)
    print(fileName, " text successful")
    m3u8Url = getM3U8Url(url)
    downloadTs(m3u8Url, fileName)

In [29]:
total = 0
for i in range(1, 7):
    url = f'https://tv.chinacourt.org/shipin-13.html?Video_page=6&ajax=yw1&page={i}'
    for link in getPageUrl(url):
        print(f'total {total}')
        print(link)
        downloadVideoAndText(link, str(total))
        total = total + 1

total 0
https://tv.chinacourt.org/53810.html
0  text successful
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/E15BE966953D019E2A977CDB58D733D6/media_w1231573837_0.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/E15BE966953D019E2A977CDB58D733D6/media_w1231573837_1.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/E15BE966953D019E2A977CDB58D733D6/media_w1231573837_2.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/E15BE966953D019E2A977CDB58D733D6/media_w1231573837_3.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/E15BE966953D019E2A977CDB58D733D6/media_w1231573837_4.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/E15BE966953D019E2A977CDB58D733D6/media_w1231573837_5.ts
ts 下载完成
0  ts successful
total 1
https://tv.chinacourt.org/53809.html
1  text successful
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/9E0AEEA981F858DB8848597F9D536DE0/media_w958073979_0.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/9E0AEEA981F858DB

https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_59.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_60.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_61.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_62.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_63.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_64.ts
https://newmedia.chinacourt.org/vod/play/2021/03/10/08/CE98F19D717A53E6E1F37BA3F8E03DE1/media_w685340239_65.ts
ts 下载完成
2  ts successful
total 3
https://tv.chinacourt.org/53782.html
3  text successful
https://newmedia.chinacourt.org/vod/play/2021/03/09/08/831F1544FC68799C8C0D0983613D0E30/media_w1873155507_0.ts
https://newmedia.chinac

https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_49.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_50.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_51.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_52.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_53.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_54.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_55.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w1851667407_56.ts
https://newmedia.chinacourt.org/vod/play/2021/03/09/09/515F4C4ADE4CC354A2E7BB4E9CFD4230/media_w185166740

https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_22.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_23.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_24.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_25.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_26.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_27.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_28.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_29.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w155060593

https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_96.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_97.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_98.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_99.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_100.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_101.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_102.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w1550605935_103.ts
https://newmedia.chinacourt.org/vod/play/2021/03/08/10/56A0B6652E0DF80A0D8E0CABBAD90AEE/media_w15506

...

https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_340.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_341.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_342.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_343.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_344.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_345.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_346.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370_347.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/a64baf8b801fff23e56280634761db60/media_w775766370

https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_18.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_19.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_20.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_21.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_22.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_23.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_24.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_25.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w209993189

https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_92.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_93.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_94.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_95.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_96.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_97.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_98.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w2099931893_99.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/41a8ca5ea67c89b20a75dae92cd28528/media_w209993189

https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_18.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_19.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_20.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_21.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_22.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_23.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_24.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_25.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w133446022

https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_92.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_93.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_94.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_95.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_96.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_97.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_98.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w1334460224_99.ts
https://newmedia.chinacourt.org/vod/play/2011/12/09/15/b0ec79f7dce319000d0b55b326b0b294/media_w133446022